# Data Wrangling with Penguins

* The dataset contains several measurements (features) of 344 penguins across 3 antartic islands
* It contains a species designation (the label).
* It is frequently used for learning how to develop machine learning classification models.
* This Jupyter notebook contains data science work to analyze the data and train classification models.

![TestImage](img/penguins3.JPG)

### Data Science Project Samples
There are many sample data science projects to download from the internet.
These examples are based upon a couple of projects published in Kaggle at the links below.

* https://www.kaggle.com/code/parulpandey/penguin-dataset-the-new-iris/notebook
* https://www.kaggle.com/code/mohamedharris/how-to-classify-penguins-a-beginner-s-guide


### Step 1 Connect to Db2 z/OS and review the data

In [1]:
pip install ipython-sql==0.4.1 ibm_db sqlalchemy==1.4.47 ibm_db_sa 

Note: you may need to restart the kernel to use updated packages.


In [2]:
# and import them...

import sys,os,os.path
os.environ['IBM_DB_HOME']="/Applications/dsdriver"

import ibm_db 
import ibm_db_sa 
import sqlalchemy 

In [3]:
# Now, lets take a look at the Penguins Dataset 
%load_ext sql
%sql ibm_db_sa://WMLSCORS:vesna08a@10.3.58.61:448/RDBNDW00
%sql select * from TSOLENK.PENGUINS_DATA fetch first 10 rows only


 * ibm_db_sa://WMLSCORS:***@10.3.58.61:448/RDBNDW00
Done.


id,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
1,Adelie,Torgersen,39.1,18.7,181,3750,MALE
2,Adelie,Torgersen,39.5,17.4,186,3800,FEMALE
3,Adelie,Torgersen,40.3,18.0,195,3250,FEMALE
4,Adelie,Torgersen,None,None,None,None,None
5,Adelie,Torgersen,36.7,19.3,193,3450,FEMALE
6,Adelie,Torgersen,39.3,20.6,190,3650,MALE
7,Adelie,Torgersen,38.9,17.8,181,3625,FEMALE
8,Adelie,Torgersen,39.2,19.6,195,4675,MALE
9,Adelie,Torgersen,34.1,18.1,193,3475,None
10,Adelie,Torgersen,42.0,20.2,190,4250,None


In [4]:
# A quick count of the data records by island, species and sex

%sql select island, species, sex, count(*) as count from TSOLENK.PENGUINS_DATA group by island, species, sex order by island, species, sex

 * ibm_db_sa://WMLSCORS:***@10.3.58.61:448/RDBNDW00
Done.


island,species,sex,COUNT
Biscoe,Adelie,FEMALE,22
Biscoe,Adelie,MALE,22
Biscoe,Gentoo,FEMALE,58
Biscoe,Gentoo,MALE,61
Biscoe,Gentoo,None,5
Dream,Adelie,FEMALE,27
Dream,Adelie,MALE,28
Dream,Adelie,None,1
Dream,Chinstrap,FEMALE,34
Dream,Chinstrap,MALE,34


### Step 2 Install and Import the standard data science tools you wish to use

* Libraries exist for many purposes (data access, mathematical tools, visualisation tools etc...)
* The libraries that you choose to use must be installed and imported into this python kernel (cw01 - top right).

In [5]:
pip install pandas dcor numpy scikit-learn matplotlib seaborn pycountry plotly cufflinks folium nbformat chart_studio

  Using cached pandas-2.2.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata (89 kB)
  Preparing metadata (setup.py) ... done
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached MarkupSafe-3.0.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (4.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached pandas-2.2.3-cp311-cp311-macosx_10_9_x86_64.whl (12.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 16.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 15.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 16.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 17.3 MB/s eta 0:00:00a 0:00:01
Using cached cycler-0.12.1-

In [6]:
# import the libraries you want to use

import numpy as np 
import pandas as pd 
import dcor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import os

# Visualisation libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import pycountry
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
!pip install chart_studio
import chart_studio.plotly as py
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
#py.init_notebook_mode(connected=True)

#Geographical Plotting
import folium
from folium import Choropleth, Circle, Marker
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster

#Racing Bar Chart
!pip install bar_chart_race
import bar_chart_race as bcr
from IPython.display import HTML

import nbformat 

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = 8, 5
plt.style.use("fivethirtyeight")# for pretty graphs

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Disable warnings 
# import warnings
# warnings.filterwarnings('ignore')

Matplotlib is building the font cache; this may take a moment.


### Step 3 Create and Populate a dataframe

* a DataFrame is a 2-dimensional labeled data structure with columns of potentially different types.
* It's conceptually the same as a Db2 table, but exists in the Python kernel for the data scientist to use

In [8]:
# assign a variable that contains a string of your credentials
credentials = "ibm_db_sa://WMLSCORS:vesna08a@10.3.58.61:448/RDBNDW00"

# read in your SQL query results using pandas
dataframe = pd.read_sql("""
            select species, island, bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g, sex, year 
            from TSOLENK.PENGUINS_DATA
            ORDER BY id
            """, con = credentials)

# and save the original
original = dataframe.copy() 

# return your first ten rows
dataframe.head(10)


ImportError: Using URI string without sqlalchemy installed.

In [ ]:
# Understand the fields in the dataframe

dataframe.info()


In [ ]:
# Calculate some numercial average and distribution data

dataframe.describe(include='all')

### Step 4 Start "Data Wrangling" with pandas

Data Wrangling consists of 5 main activity areas:

1. Data exploration — feature values, ranges, correlations, relationships...
2. Dealing with missing values — various strategies
3. Reshaping data — pivot tables, joins, grouping and aggregating
4. Filtering data - selection, projection etc...
5. Other — Making descriptive columns, element-wise conditional operations

In [ ]:
# Count the number of each species in the labelled dataset

dataframe['species'].value_counts()

In [ ]:
# or represent the counts graphically

dataframe['species'].value_counts().iplot(kind='bar')

In [ ]:
# Create a filtered dataframe for exploring ranges of data values for different species of penguin

df1 = dataframe[['species', 'bill_length_mm', 'bill_depth_mm','flipper_length_mm']]
sns.boxplot(data=df1, width=0.5,fliersize=5)

In [ ]:
# Now, focus on bill length by species and sex.

def box(f):
    sns.boxplot(y = f, x = 'species', hue = 'sex',data = dataframe)
    plt.title(f)
    plt.show() 

box('bill_length_mm')

In [ ]:
# Each of the features are helpful in differentiating between species. Combining multiple features can expose stronger data patterns.

sns.pairplot(df1, hue="species", height=3,diag_kind="hist")

In [ ]:
# we can zoom in on particular pairplots

sns.FacetGrid(df1, hue="species", height=8) \
   .map(plt.scatter, "bill_length_mm", "flipper_length_mm") \
   .add_legend()

In [ ]:
# another visualisation tool is the violin plot

ax = sns.violinplot(x="species", y="flipper_length_mm", data=df1,height=8)

In [ ]:
# and we can plot distribution curves

sns.FacetGrid(df1, hue="species", height=6,) \
   .map(sns.kdeplot, "flipper_length_mm",fill=True) \
   .add_legend()

### Step 5 : Cleaning the Data

The previous section showed a very small subset of data wrangling techniques to allow the data scientist to understand the dataset, see patterns in the data, and start to form ideas about which model types may be best suited to the task of creating a classification model.

Before fitting and training a model, a data pipeline should be built to cleanse and transform the data into a suitable form. Data cleansing work may include
* identifying null or empty values, and deciding on a technique to handle them
* eliminating data features that don't have a big impact on the classification of records
* converting string datatypes into numerical types
* Normalising the data values into ranges with good distribution of values
etc...

In [ ]:
# Import the libraries for 5 model types

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
# Eyeball the first 10 rows. We can see nulls and missing values.

dataframe.head(10)

In [ ]:
# Check the counts of Null values

dataframe.isnull().sum()

In [ ]:
# Create a new dataframe and choose statistical means for missing values
# in the example below we just choose the statistical mean value for the 4 measurement data features, and the modal value for sex data feature.

new_df = original.copy()

new_df['bill_length_mm'].fillna(np.mean(original['bill_length_mm']), inplace = True)
new_df['bill_depth_mm'].fillna(np.mean(original['bill_depth_mm']), inplace = True)
new_df['flipper_length_mm'].fillna(np.mean(original['flipper_length_mm']), inplace = True)
new_df['body_mass_g'].fillna(np.mean(original['body_mass_g']), inplace = True)
new_df['sex'].fillna(original['sex'].mode()[0], inplace = True)

In [ ]:
# Verify that we have assigned values to all the missing data features

new_df.head(10)

In [ ]:
# and check that the null values are missing

new_df.isnull().sum()

In [ ]:
# skewness is a statistical measure of asymmetry from a normal distribution. Skewness between -0.5 and +0.5 is considered symmetrical.
# this dataset does not suffer from skewing

print('Skewness of numeric variables')
print('-' * 35)

for i in new_df.select_dtypes(['int64', 'float64']).columns.tolist():
    print(i, ' : ',new_df[i].skew())

In [ ]:
# Now take a look at the range of actual values for the data features.new_df

new_df.describe()

In [ ]:
# the goal of data normalization is to ensure that data is similar across all records, to make the model more efficient
# The function below is used to normalse the numerical data features between the values of -1 and +1.
# See how it compresses the data values. (aka "squishification")

from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

new_df['bill_length_mm'] = mms.fit_transform(new_df['bill_length_mm'].values.reshape(-1, 1))
new_df['bill_depth_mm'] = mms.fit_transform(new_df['bill_depth_mm'].values.reshape(-1, 1))
new_df['flipper_length_mm'] = mms.fit_transform(new_df['flipper_length_mm'].values.reshape(-1, 1))
new_df['body_mass_g'] = mms.fit_transform(new_df['body_mass_g'].values.reshape(-1, 1))
new_df['sex'].fillna(original['sex'].mode()[0], inplace = True)

new_df.describe()

In [ ]:
new_df.head()

In [ ]:
# Now lets create dummy values for sex and island fields

new_df_dummy = pd.get_dummies(new_df, columns = ['sex', 'island'], drop_first = True) 

new_df_dummy.head(10)




In [ ]:
# Now lets assign numerical values for the 3 species. First we check the unique text values that exist.

new_df_dummy['species'].unique()

In [ ]:
#Next, we replace those values with numerical surrogate values.

new_df_dummy['species'].replace(['Adelie                        ',
                                  'Chinstrap                     ',
                                    'Gentoo                        '],
                                     [0, 1, 2], inplace = True)

new_df_dummy.head(10)

In [ ]:


sns.heatmap(new_df_dummy.corr(), annot = True, cmap = 'Blues')



In [ ]:
# Create 2 sets of data, with and without the label

X = new_df_dummy.drop(columns = ['species'])
Y = new_df_dummy['species']

In [ ]:
# Split the Data 75% training and 25% test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 123)

In [ ]:
# Try with a Logical Regression Model

LR = LogisticRegression()
LR.fit(X_train, Y_train)

pred = LR.predict(X_test)

In [ ]:
# Check the Accuracy and F1 Score of the trained model

print('Accuracy : ', accuracy_score(Y_test, pred))
print('F1 Score : ', f1_score(Y_test, pred, average = 'weighted')) 
print('Precision : ', precision_score(Y_test, pred , average = 'weighted'))
print('Recall : ', recall_score(Y_test, pred, average = 'weighted'))

In [ ]:
# Try some other candidate models 

models = []
models.append(('LR', LogisticRegression()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('kNN', KNeighborsClassifier()))
models.append(('SVC', SVC()))